# Buy and Hold Strategy

This notebook demonstrates a simple buy-and-hold strategy:
- Buy SPY (S&P 500 ETF) on the first day
- Hold until the end
- Track performance with FlightLog

## Setup

In [ ]:
# Register Sharadar bundle (required for Jupyter notebooks)
from zipline.data.bundles import register
from zipline.data.bundles.sharadar_bundle import sharadar_bundle

register('sharadar', sharadar_bundle(tickers=None, incremental=True, include_funds=True))
print("✓ Sharadar bundle registered")

In [ ]:
import logging
import pandas as pd
import matplotlib.pyplot as plt

from zipline import run_algorithm
from zipline.api import (
    order,
    order_target_percent,
    symbol,
    record,
    schedule_function,
    date_rules,
    time_rules,
)
from zipline.utils.progress import enable_progress_logging
from zipline.utils.flightlog_client import enable_flightlog

# Enable logging
logging.basicConfig(level=logging.INFO, force=True)

# Connect to FlightLog (optional - remove if not using)
# enable_flightlog(host='flightlog', port=9020)

# Enable progress tracking
enable_progress_logging(algo_name='Buy-and-Hold', update_interval=10)

## Strategy Implementation

Buy SPY on the first day and hold.

In [ ]:
def initialize(context):
    """
    Called once at the start of the algorithm.
    """
    # Set the stock to trade
    context.spy = symbol('SPY')
    
    # Track if we've made our initial purchase
    context.has_bought = False
    
    logging.info(f"Strategy initialized. Trading: {context.spy.symbol}")


def handle_data(context, data):
    """
    Called every day (or bar).
    """
    # Buy on the first day
    if not context.has_bought:
        # Get current price
        price = data.current(context.spy, 'price')
        
        # Invest 100% of portfolio in SPY
        order_target_percent(context.spy, 1.0)
        
        context.has_bought = True
        logging.info(f"Bought {context.spy.symbol} at ${price:.2f}")
    
    # Record daily values
    record(
        SPY=data.current(context.spy, 'price'),
        portfolio_value=context.portfolio.portfolio_value
    )


def analyze(context, perf):
    """
    Called after the backtest completes.
    """
    logging.info("\n" + "="*60)
    logging.info("BACKTEST RESULTS")
    logging.info("="*60)
    
    # Calculate returns
    start_value = perf['portfolio_value'].iloc[0]
    end_value = perf['portfolio_value'].iloc[-1]
    total_return = (end_value - start_value) / start_value * 100
    
    logging.info(f"Start date: {perf.index[0].date()}")
    logging.info(f"End date: {perf.index[-1].date()}")
    logging.info(f"")
    logging.info(f"Starting portfolio value: ${start_value:,.2f}")
    logging.info(f"Ending portfolio value: ${end_value:,.2f}")
    logging.info(f"Total return: {total_return:.2f}%")
    logging.info(f"")
    logging.info(f"Number of transactions: {len(perf[perf['transactions'].apply(len) > 0])}")
    logging.info("="*60)

## Run Backtest

Run the strategy from 2020 to 2023.

In [ ]:
# Run the backtest
results = run_algorithm(
    start=pd.Timestamp('2020-01-01'),
    end=pd.Timestamp('2023-12-31'),
    initialize=initialize,
    handle_data=handle_data,
    analyze=analyze,
    capital_base=100000,  # Start with $100,000
    data_frequency='daily',
    bundle='sharadar',
)

## Analyze Results

Plot portfolio value over time.

In [ ]:
# Plot portfolio value
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Portfolio value
axes[0].plot(results.index, results['portfolio_value'], linewidth=2)
axes[0].set_ylabel('Portfolio Value ($)', fontsize=12)
axes[0].set_title('Buy and Hold: SPY (2020-2023)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Format y-axis as currency
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# SPY price
axes[1].plot(results.index, results['SPY'], linewidth=2, color='orange')
axes[1].set_ylabel('SPY Price ($)', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)
print(f"Total Return: {((results['portfolio_value'].iloc[-1] / results['portfolio_value'].iloc[0]) - 1) * 100:.2f}%")
print(f"Max Portfolio Value: ${results['portfolio_value'].max():,.2f}")
print(f"Min Portfolio Value: ${results['portfolio_value'].min():,.2f}")
print(f"Final Portfolio Value: ${results['portfolio_value'].iloc[-1]:,.2f}")
print("="*60)

## View Transactions

See when trades were executed.

In [ ]:
# Extract transactions
transactions = []
for date, row in results.iterrows():
    if len(row['transactions']) > 0:
        for txn in row['transactions']:
            transactions.append({
                'Date': date,
                'Symbol': txn['sid'].symbol,
                'Amount': txn['amount'],
                'Price': txn['price'],
                'Value': txn['amount'] * txn['price']
            })

if transactions:
    txn_df = pd.DataFrame(transactions)
    print("\nTransactions:")
    print(txn_df.to_string(index=False))
else:
    print("\nNo transactions executed.")

## Next Steps

- Try different start/end dates
- Change the stock (e.g., 'AAPL', 'MSFT', 'QQQ')
- Compare with other strategies
- See `03_moving_average_crossover.ipynb` for a more active strategy
- See `04_pyfolio_analysis.ipynb` for detailed performance analysis